In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from model import *
from pickle_cache import PickleCache
import pandas as pd
from experiment_widgets import VariableArithmeticMemoryExperiment
import json
from random import sample, randint, choices
import string
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set()

pcache = PickleCache()

In [3]:
all_names = string.ascii_lowercase
all_operators = ['+', '-']

def generate_trial(N):
    names = sample(all_names, k=N)
    variables = [
        {'variable': names[i], 'value': randint(1, 9)}
        for i in range(N)
    ]

    expr_var_order = sample(variables, k=len(variables))
    operators = choices(all_operators, k=N-1)
    expr_list = []
    expr_value = expr_var_order[0]['value']
    for i in range(N):
        if i > 0:
            op = operators[i-1]
            expr_value = eval(f"{expr_value} {op} {expr_var_order[i]['value']}")
            expr_list.append(op)
        expr_list.append(expr_var_order[i]['variable'])
    expr = ' '.join(expr_list)
    
    return {
        'variables': variables, 
        'expression': expr,
        'expression_value': expr_value,
        'presentation_time': 500 + N * 1500
    }

def exp_name(N_var, N_trials):
    return f'vararithmem_{N_var}_{N_trials}'

In [4]:
N_var = 3
N_trials = 10


experiment = {
    'trials': [generate_trial(N_var) for _ in range(N_trials)],
    'between_trials_time': 1000
}

exp = VariableArithmeticMemoryExperiment(experiment=json.dumps(experiment), results='[]')

def on_result_change(_):
    pcache.set(exp_name(N_var, N_trials), {
        'experiment': experiment,
        'results': json.loads(exp.results)
    })
    
#exp.observe(on_result_change)

exp

VariableArithmeticMemoryExperiment(experiment='{"trials": [{"variables": [{"variable": "r", "value": 6}, {"var…

In [5]:
def process_results(N_var, N_trials):
    data = pcache.get(exp_name(N_var, N_trials))
    experiment = data['experiment']
    results = data['results']
    
    correct = 0
    for (trial, result) in zip(experiment['trials'], results):
        try:
            response = int(result['response'])
        except ValueError:
            continue
            
        if trial['expression_value'] == response:
            correct += 1

    return {
        'N_var': N_var,
        'Correct': correct / N_trials,
    }

In [6]:
all_exp = [2, 3, 4, 5]
results = pd.DataFrame([process_results(N_var, 10) for N_var in all_exp])

def plot_results(results):
    results = results.set_index('N_var')
    ax = results.plot.bar()
    ax.set_ylabel('Fraction of responses over 10 trials')
    return ax

    #plt.savefig('var_arith_mem.png', dpi=300)
    
plot_results(results)

In [7]:
def model(trial):
    wm = WorkingMemory()
    
    for var in trial['variables']:
        wm.store_var_val(var['variable'], var['value'])
        
    expr = parse(trial['expression']).statements[0].value
    return trace_expr(expr, wm) == trial['expression_value']

df = []
for N_var in all_exp:
    #data = pcache.get(exp_name(4, N_trials))
    #trials = data['experiment']['trials']
    trials = [generate_trial(N_var) for _ in range(1000)]
    correct = 0
    for trial in trials:
        correct += 1 if model(trial) else 0
    df.append({
        'N_var': N_var,
        'correct': correct / len(trials)
    })
plot_results(pd.DataFrame(df))

TypeError: guess_behavior() missing 1 required positional argument: 'wm'